### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 7
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 7, 7), datetime.date(2023, 7, 6))

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-07-07
yesterday: 2023-07-06


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(35, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(187, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(35, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(35, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,BDMS,-17,XXX,SET50,1000,24.0,26.75,-0.5,-1.834862,0,2022-12-22
B,BEM,-10,52WL,SET50,2500,7.8,8.3,-0.05,-0.598802,0,2022-12-22
B,BH,-82,5pct,SET50,200,169.0,220.0,-3.0,-1.345291,0,2022-12-22
B,CPNREIT,-8,5pct,SET,5000,10.2,11.0,0.2,1.851852,1,2023-05-23
B,DIF,-3,5pct,SET,10000,10.6,10.9,0.0,0.0,1,2023-05-15
B,EA,-4,52WL,SET50,1200,53.0,54.0,-0.25,-0.460829,2,2022-12-22
B,GVREIT,-3,5pct,SET,10000,8.1,8.25,-0.05,-0.60241,1,2023-05-26
B,JMT,-13,RD35pct,SET50,1200,33.0,36.25,-0.25,-0.684932,0,2023-04-11
B,KKP,-3,,SET100,500,58.0,58.75,-0.75,-1.260504,1,2023-04-28
B,KTB,-49,,SET50,1500,14.3,19.2,0.0,0.0,0,2023-04-20
B,KTC,-4,RD10pct,SET50,600,45.0,46.0,-0.25,-0.540541,2,2022-12-22
B,LANNA,-12,,SET,1500,13.8,15.0,0.1,0.671141,0,2023-03-18
B,MCS,-9,RD60pct,SET,4000,6.0,6.45,0.0,0.0,1,2023-03-17
B,NER,-5,RD40pct,SET,5000,4.4,4.5,-0.04,-0.881057,1,2023-04-20
B,ORI,-1,52WL,SET100,3000,10.0,10.1,0.0,0.0,1,2023-05-08
B,PT

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
4,B,DIF,-3,5pct,SET,10000,10.60,10.90,0.00,0.00,1,2023-05-15,106000.00
6,B,GVREIT,-3,5pct,SET,10000,8.10,8.25,-0.05,-0.60,1,2023-05-26,81000.00
8,B,KKP,-3,NaN,SET100,500,58.00,58.75,-0.75,-1.26,1,2023-04-28,29000.00
14,B,ORI,-1,52WL,SET100,3000,10.00,10.10,0.00,0.00,1,2023-05-08,30000.00
15,B,PTG,-3,NaN,SET100,1800,11.40,11.70,0.00,0.00,1,2022-12-22,20520.00
19,B,SYNEX,-3,RND60pct,SET,2000,11.70,12.00,0.00,0.00,1,2023-03-09,23400.00
23,B,WHAIR,-3,RD20pct,SET,10000,6.90,7.05,0.00,0.00,2,2022-02-02,69000.00
33,S,TFFIF,2,DOS,SET,10000,7.70,7.60,0.00,0.00,2,2023-05-31,77000.00


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
5,B,EA,-4,52WL,SET50,1200,53.00,54.00,-0.25,-0.46,2,2022-12-22,63600.00
10,B,KTC,-4,RD10pct,SET50,600,45.00,46.00,-0.25,-0.54,2,2022-12-22,27000.00
17,B,SENA,-5,RD35pct,SET,7000,2.88,2.98,0.00,0.00,2,2023-05-11,20160.00
23,B,WHAIR,-3,RD20pct,SET,10000,6.90,7.05,0.00,0.00,2,2022-02-02,69000.00
33,S,TFFIF,2,DOS,SET,10000,7.70,7.60,0.00,0.00,2,2023-05-31,77000.00
34,S,WHART,5,5pct,SET,10000,11.00,10.50,0.00,0.00,2,2022-02-02,110000.00


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
3,B,CPNREIT,-8,5pct,SET,5000,10.20,11.00,0.20,1.85,1,2023-05-23,51000.00
4,B,DIF,-3,5pct,SET,10000,10.60,10.90,0.00,0.00,1,2023-05-15,106000.00
6,B,GVREIT,-3,5pct,SET,10000,8.10,8.25,-0.05,-0.60,1,2023-05-26,81000.00
8,B,KKP,-3,NaN,SET100,500,58.00,58.75,-0.75,-1.26,1,2023-04-28,29000.00
12,B,MCS,-9,RD60pct,SET,4000,6.00,6.45,0.00,0.00,1,2023-03-17,24000.00
13,B,NER,-5,RD40pct,SET,5000,4.40,4.50,-0.04,-0.88,1,2023-04-20,22000.00
14,B,ORI,-1,52WL,SET100,3000,10.00,10.10,0.00,0.00,1,2023-05-08,30000.00
15,B,PTG,-3,NaN,SET100,1800,11.40,11.70,0.00,0.00,1,2022-12-22,20520.00
19,B,SYNEX,-3,RND60pct,SET,2000,11.70,12.00,0.00,0.00,1,2023-03-09,23400.00
21,B,TMT,-9,RD30pct,SET,3000,6.95,7.40,0.00,0.00,1,2023-04-18,20850.00


In [17]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,BDMS,-17,XXX,SET50,1000,24.00,26.75,-0.50,-1.83,0,2022-12-22,24000.00
1,B,BEM,-10,52WL,SET50,2500,7.80,8.30,-0.05,-0.60,0,2022-12-22,19500.00
2,B,BH,-82,5pct,SET50,200,169.00,220.00,-3.00,-1.35,0,2022-12-22,33800.00
7,B,JMT,-13,RD35pct,SET50,1200,33.00,36.25,-0.25,-0.68,0,2023-04-11,39600.00
9,B,KTB,-49,NaN,SET50,1500,14.30,19.20,0.00,0.00,0,2023-04-20,21450.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,S,BCH,50,5pct,SET100,3000,22.40,17.40,0.00,0.00,0,2023-05-09,67200.00
28,S,ICHI,14,5pct,SET,1500,15.00,13.60,0.30,2.26,0,2022-12-22,22500.00
29,S,IVL,18,5pct,SET50,2400,37.75,33.25,-0.50,-1.48,0,2023-05-26,90600.00
31,S,JMART,16,5pct,SET50,3600,17.90,16.30,-0.10,-0.61,0,2023-04-11,64440.00


In [18]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [19]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [20]:
df['percent'] = df['%change']

In [21]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [22]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
24,S,AH,13,5pct,SET,1200,39.00,35.75,0.75,2.14,0,2022-12-22,46800.00,2.14
28,S,ICHI,14,5pct,SET,1500,15.00,13.60,0.30,2.26,0,2022-12-22,22500.00,2.26


In [23]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
14,B,ORI,-1,52WL,SET100,3000,10.00,10.10,0.00,0.00,1,2023-05-08,30000.00,0.00
23,B,WHAIR,-3,RD20pct,SET,10000,6.90,7.05,0.00,0.00,2,2022-02-02,69000.00,0.00
4,B,DIF,-3,5pct,SET,10000,10.60,10.90,0.00,0.00,1,2023-05-15,106000.00,0.00
6,B,GVREIT,-3,5pct,SET,10000,8.10,8.25,-0.05,-0.60,1,2023-05-26,81000.00,-0.60
19,B,SYNEX,-3,RND60pct,SET,2000,11.70,12.00,0.00,0.00,1,2023-03-09,23400.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,B,TASCO,-25,XXX,SET,1500,15.60,18.10,-0.20,-1.09,0,2022-12-22,23400.00,-1.09
16,B,PTTEP,-32,NaN,SET50,200,134.50,150.50,-0.50,-0.33,0,2023-02-14,26900.00,-0.33
22,B,TTB,-45,52WL,SET50,20000,1.09,1.54,0.00,0.00,0,2023-04-18,21800.00,0.00
9,B,KTB,-49,NaN,SET50,1500,14.30,19.20,0.00,0.00,0,2023-04-20,21450.00,0.00


### Sell recently purchased stocks if profitable

In [24]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(108, 4)

In [25]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [26]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(105, 8)

In [27]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
4,BBL,2023-07-07,400,155.00,156.00,400.00,0.00,0.65
95,TFFIF,2023-07-06,10000,7.55,7.60,500.00,0.00,0.66
107,WHART,2022-05-17,10000,10.50,10.50,0.00,0.00,0.00


### Profit per Day

In [28]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 7
ORDER BY sells.date DESC, name


In [29]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,JMART,2023,7,17.90,17.00,0.90,"1,800","32,220.00","30,600.00","1,620.00",5.29,13


In [30]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [31]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,JMART,2023,7,17.90,17.00,0.90,"1,800","32,220.00","30,600.00","1,620.00",5.29,13,0.407240


In [32]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.41%
